In [8]:
# import GNNAgg as agg
# import GNNArch as arc
# import GNNLayers as layers
# import GNNModel as models
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))


In [ ]:
dataset = Planetoid(root="../models/data/", name="Cora", transform=NormalizeFeatures())

In [ ]:
from torch_geometric.loader import DataLoader
data_loader = DataLoader(dataset, batch_size=10)

In [ ]:
i = next(iter(data_loader))

In [ ]:
i.batch.shape

In [ ]:
net = models.GraphGNNModels(
    in_channel=dataset.num_features,
    edge_channel=dataset.num_edge_features,
    out_channel=dataset.num_classes,
    embedding_layer_num=3,
    architecture_name='IterGNN',
    layer_num=10,
    layer_name='GCNConv',
    hidden_size=64,
    confidence_layer_num=3
)
print(net)

In [ ]:
data = dataset[0]

In [ ]:
import numpy as np
def running_test(net, test_data_loader, device,
                 running_metric_name_list, test_loss_hist,
                 epoch, step, global_step):
    test_metric_list, _ = evaluate(net, test_data_loader, device,
                                parallel_flag=False,
                                metric_name_list=running_metric_name_list)
    for mn in running_metric_name_list:
        test_loss_hist[mn].append(np.mean(test_metric_list[mn]))

In [ ]:
from torch_geometric.loader import DataLoader
global_step = 0
layer_num_hist, loss_hist = [], []
running_metric_name_list = ["relative loss", "mse loss"]
test_loss_hist, test_gen_loss_hist = [{mn:[] for mn in running_metric_name_list} for _ in range(2)]
epoch_num=10
data_loader = DataLoader(dataset)
import torch.nn.functional as F
from torch.optim import Adam
optimizer = Adam(net.parameters(), lr=0.001)

for epoch in range(epoch_num+1):
        print("111")
        net.train()
        print(data_loader)
        
        for step,data in enumerate(data_loader):
            data = data.to("cpu")
            global_step += 1
            pred = net(data)
            y = data.y
            print(y.size())
            loss = F.mse_loss(pred.reshape(y.size()), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_hist.append(loss.item())
            layer_num_hist.append(layer_num)
        running_test(net, data_loader, "cpu",
                     running_metric_name_list, test_loss_hist,
                     epoch, step, global_step)

In [5]:
import torch

aa = torch.rand(5,1)
batch = torch.tensor([0,0,0,0,1,1,1,1,2,2,2,3,3,3,3,4,4,4])
print(aa)
print(batch)
print(aa[batch])

tensor([[0.7828],
        [0.5856],
        [0.1311],
        [0.2514],
        [0.3540]])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4])
tensor([[0.7828],
        [0.7828],
        [0.7828],
        [0.7828],
        [0.5856],
        [0.5856],
        [0.5856],
        [0.5856],
        [0.1311],
        [0.1311],
        [0.1311],
        [0.2514],
        [0.2514],
        [0.2514],
        [0.2514],
        [0.3540],
        [0.3540],
        [0.3540]])


In [7]:
import torch
from torch_geometric.nn import global_max_pool
from OtherLayers import MLP
mlp = MLP(size_list=[7,1])
aa = torch.rand(5,7)
batch = torch.tensor([0,0,0,3,4])
bbb = global_max_pool(aa, batch)
mlp(bbb)

tensor([[ 0.3160],
        [-0.0029],
        [-0.0029],
        [ 0.1488],
        [-0.0008]], grad_fn=<LeakyReluBackward0>)

In [2]:
from GNNAgg import ReadoutLayers
from torch_geometric.data import Data
rr = ReadoutLayers('Max')

In [3]:
data1 = Data(x=aa, num_graphs=1)
print(data1.batch)

None


In [10]:
data = Data(x=aa, batch=batch, num_graphs=1)
ppp = rr(data)
qqq = mlp(ppp)
qqq.shape

torch.Size([5, 1])

In [ ]:
data.batch is not None

In [7]:
from OtherLayers import LobsterEncoder
import torch.nn as nn
aa = LobsterEncoder(in_channel=5, out_channel=1, layer_num=3, last_activation=nn.Identity)
aa

LobsterEncoder(
  (module): Sequential(
    (0): Sequential(
      (0): Linear(in_features=5, out_features=3, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=3, out_features=2, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=2, out_features=1, bias=True)
      (1): Identity()
    )
  )
)

In [28]:
from sklearn.model_selection import StratifiedKFold
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import OneHotDegree
from torch_geometric.utils import degree
import torch
from main.Planetoid_utils import add_noise
dataset = TUDataset(root='data/', name="MUTAG")
# running_max = 0
# for data in dataset:
#     deg = max(degree(data.edge_index[0])).item()
#     if deg > running_max:
#         running_max = deg
# print(running_max)
dataset[0].x[0]


tensor([1., 0., 0., 0., 0., 0., 0.])

In [33]:
import numpy as np
torch.tensor(np.random.normal(0, 1, dataset[0].x.size())) + dataset[0].x

tensor([[ 1.4989,  0.8419,  1.7994, -1.4881, -0.2392, -0.9777,  0.6797],
        [ 0.4956, -0.0920,  0.2642,  0.3930,  0.9706,  1.4442, -0.1691],
        [ 1.4467,  0.5989, -0.6451, -0.2349,  0.6676, -2.0253,  0.1696],
        [ 2.5653, -0.6892, -0.9036,  1.6700, -0.8507, -0.3002,  0.5359],
        [-1.6184, -1.9339, -0.6903,  0.5359,  0.2972, -0.4479,  3.0432],
        [ 1.5381, -0.0292,  1.4400,  0.6650,  1.8275,  0.2086, -1.1616],
        [-1.4297,  0.2855, -0.0602,  0.3554,  0.1014, -0.7319, -2.1213],
        [ 0.7748,  0.8108,  0.7299,  1.0317,  0.8967, -0.6605, -0.2472],
        [ 0.1302, -2.6237,  0.2541,  0.3003,  0.0243, -0.0567, -1.4989],
        [ 1.2966,  0.9576, -0.8157,  1.6909, -0.1964,  1.0285,  0.5211],
        [-0.7091, -0.1175, -0.7209,  0.6056, -0.6699,  0.6292,  0.3205],
        [ 1.8076, -0.4284,  0.2154,  0.9747,  0.2471,  0.0193,  1.0326],
        [ 0.4988,  0.3517, -0.6324,  0.1111,  0.5408,  0.7150,  0.1840],
        [ 2.4299,  1.3979, -0.0658, -0.8267, -0.681

In [27]:
for graph in dataset:
    graph = add_noise(graph, 0.5)
dataset[111].x

tensor([[10.,  6.,  8., 10.,  8.,  8.,  4.],
        [ 9.,  9.,  9.,  7.,  7.,  6.,  7.],
        [ 5.,  8.,  6., 10.,  8.,  7.,  6.],
        [ 8.,  7., 10., 10.,  8.,  8.,  9.],
        [ 6.,  6.,  5., 12., 11.,  7.,  5.],
        [ 9.,  7.,  4.,  7.,  4., 10.,  9.],
        [10.,  9.,  2.,  6.,  7.,  5.,  9.],
        [ 6.,  7.,  9.,  9.,  8.,  6.,  5.],
        [ 7.,  8.,  6.,  8.,  5.,  6.,  9.],
        [ 6.,  8.,  6.,  6.,  8., 10.,  7.],
        [ 6.,  8.,  7.,  9.,  6.,  6.,  7.],
        [11., 12.,  8.,  9., 11.,  9.,  8.],
        [ 6., 12.,  8., 12.,  6.,  8.,  8.],
        [ 7.,  9., 10.,  8.,  7.,  5.,  7.],
        [ 4.,  5.,  7.,  8.,  7.,  9.,  7.]])

In [12]:
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='data/', name="IMDB-MULTI", transform=OneHotDegree(88))
train_loader = DataLoader(dataset, batch_size=32)
for data in train_loader:
    print(data.edge_index)

tensor([[  0,   0,   0,  ..., 421, 421, 421],
        [  1,   2,   3,  ..., 418, 419, 420]])
tensor([[  0,   0,   0,  ..., 484, 484, 484],
        [  3,   5,   6,  ..., 481, 482, 483]])
tensor([[  0,   0,   0,  ..., 494, 494, 494],
        [  1,   3,   6,  ..., 491, 492, 493]])
tensor([[  0,   0,   0,  ..., 408, 408, 408],
        [  1,   2,   3,  ..., 404, 405, 407]])
tensor([[  0,   0,   0,  ..., 404, 404, 404],
        [  1,  12,  14,  ..., 401, 402, 403]])
tensor([[  0,   0,   0,  ..., 468, 468, 468],
        [  1,   2,   3,  ..., 462, 464, 467]])
tensor([[  0,   0,   0,  ..., 420, 420, 420],
        [  5,   7,  10,  ..., 410, 412, 413]])
tensor([[  0,   0,   0,  ..., 455, 455, 455],
        [  1,   2,   3,  ..., 452, 453, 454]])
tensor([[  0,   0,   0,  ..., 545, 545, 545],
        [  1,   2,   4,  ..., 539, 540, 543]])
tensor([[  0,   0,   0,  ..., 379, 379, 379],
        [  1,   5,   7,  ..., 373, 374, 375]])
tensor([[  0,   0,   0,  ..., 494, 494, 494],
        [  3,   6,   8, 

In [55]:
import torch.nn as nn
a = nn.Sequential(
    nn.Linear(10,12),
    nn.ReLU()
)
for layer in a:
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

In [73]:
import numpy as np
a = np.array([[1,2],[3,4]])
a[:,1]

array([2, 4])

In [20]:
from torch_geometric.nn import JumpingKnowledge
JumpingKnowledge(None)

AttributeError: 'NoneType' object has no attribute 'lower'

In [23]:
import numpy as np

a = []

a = np.concatenate((a, [1,2]))
a

array([1., 2.])

In [25]:
a = [False, True, True]
sum(a)

2

In [1]:
aa = {
    'bb': 10
}

In [2]:
print(aa)

{'bb': 10}


In [3]:
cc = {
    "cc": 12
}
aa.update(cc)

In [4]:
aa

{'bb': 10, 'cc': 12}

In [15]:
print("a={:.4f}".format(0.1))

a=    0.1000


In [7]:
import numpy as np
a = np.array([[0,1], [1,2]])
a[:,1]

array([1, 2])